In [1]:
from streaming import StreamingDataset

/home/zack/llm-foundry/env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
domains = {'c4': {'remote': 'oci://mosaicml-internal-dataset-c4/sem-dedupe/23-04-05/dataset/pretokencat/EleutherAI-gpt-neox-20b/0pt8/',
  'split': 'train'},
 'markdown': {'remote': 'oci://mosaicml-internal-datasets/stack-split-neox/markdown/',
  'split': None},
 'mc4': {'remote': 'oci://mosaicml-internal-dataset-mc4/filtered/en/filter2-gptneox-tok/',
  'split': None},
 'redpajama': {'remote': 'oci://mosaicml-internal-dataset-red-pajama/cc/gptneox-tok/',
  'split': None},
 'redpajama-arxiv': {'remote': 'oci://mosaicml-internal-dataset-red-pajama/arxiv/gptneox-tok/',
  'split': None},
 'redpajama-books': {'remote': 'oci://mosaicml-internal-dataset-red-pajama/books/gptneox-tok/',
  'split': None},
 'redpajama-stackexchange': {'remote': 'oci://mosaicml-internal-dataset-red-pajama/stackexchange/gptneox-tok/',
  'split': None},
 'redpajama-wiki': {'remote': 'oci://mosaicml-internal-dataset-red-pajama/wikipedia/gptneox-tok/en/',
  'split': None},
 's2': {'remote': 'oci://mosaicml-internal-datasets/s2orc/base/preconcat-gpt_neox/',
  'split': 'train'},
 'stack': {'remote': 'oci://mosaicml-internal-datasets/stack-split-neox/',
  'split': None}
}


In [36]:
raw_domain_samples = {}
for domain, domain_info in domains.items():
    raw_domain_samples[domain] = 2048 * StreamingDataset(remote=domain_info["remote"], local="/mnt/datateam/zack/tmp/", split=domain_info["split"]).size
total_samples = sum(raw_domain_samples.values())
raw_domain_proportions = {k: f"{(v / total_samples):.4f}" for k, v in raw_domain_samples.items()}

In [37]:
raw_domain_proportions

{'c4': '0.0245',
 'markdown': '0.0261',
 'mc4': '0.5903',
 'redpajama': '0.2145',
 'redpajama-arxiv': '0.0069',
 'redpajama-books': '0.0064',
 'redpajama-stackexchange': '0.0050',
 'redpajama-wiki': '0.0012',
 's2': '0.0119',
 'stack': '0.1132'}

In [38]:
total_samples

4096097759232

In [39]:
available_tokens = 13_000_000_000
reduction_factor = available_tokens / total_samples
domain_subsample_info = {}
for domain in domains:
    domain_subsample_info[domain] = {
        "truncate_samples": int(reduction_factor * raw_domain_samples[domain] / 2048),
        "download_remote": domains[domain]["remote"],
        "download_split": domains[domain]["split"],
        "upload_remote": f"oci://mosaicml-internal-dataset-mpt-oversample/13B-available-tokens/{domain}/train"
    }
print(domain_subsample_info)

{'c4': {'truncate_samples': 155613, 'download_remote': 'oci://mosaicml-internal-dataset-c4/sem-dedupe/23-04-05/dataset/pretokencat/EleutherAI-gpt-neox-20b/0pt8/', 'download_split': 'train', 'upload_remote': 'oci://mosaicml-internal-dataset-mpt-oversample/13B-available-tokens/c4/train'}, 'markdown': {'truncate_samples': 165923, 'download_remote': 'oci://mosaicml-internal-datasets/stack-split-neox/markdown/', 'download_split': None, 'upload_remote': 'oci://mosaicml-internal-dataset-mpt-oversample/13B-available-tokens/markdown/train'}, 'mc4': {'truncate_samples': 3747115, 'download_remote': 'oci://mosaicml-internal-dataset-mc4/filtered/en/filter2-gptneox-tok/', 'download_split': None, 'upload_remote': 'oci://mosaicml-internal-dataset-mpt-oversample/13B-available-tokens/mc4/train'}, 'redpajama': {'truncate_samples': 1361326, 'download_remote': 'oci://mosaicml-internal-dataset-red-pajama/cc/gptneox-tok/', 'download_split': None, 'upload_remote': 'oci://mosaicml-internal-dataset-mpt-oversamp

In [40]:
check_total_samples = sum([v["truncate_samples"] for v in domain_subsample_info.values()])
check_total_samples * 2048

12999991296

In [41]:
check_domain_proportions = {k: v["truncate_samples"] / check_total_samples for k, v in domain_subsample_info.items()}
check_domain_proportions

{'c4': 0.024515049029152827,
 'markdown': 0.026139271655093884,
 'mc4': 0.5903151275463746,
 'redpajama': 0.21446134728242822,
 'redpajama-arxiv': 0.006858914130768353,
 'redpajama-books': 0.006353057792078079,
 'redpajama-stackexchange': 0.005015555358107218,
 'redpajama-wiki': 0.0011905189509443808,
 's2': 0.01192629967742403,
 'stack': 0.11322485857762839}

In [42]:
list(domains.keys())

['c4',
 'markdown',
 'mc4',
 'redpajama',
 'redpajama-arxiv',
 'redpajama-books',
 'redpajama-stackexchange',
 'redpajama-wiki',
 's2',
 'stack']